In [74]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [75]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [76]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
db_path = "C:/Users/kcalo/Desktop/bootcamp/sqlalchemy-challenge/SurfsUp/Resources/hawaii.sqlite"
print("Database exists:", os.path.exists(db_path))

In [78]:
# create engine to hawaii.sqlite
engine = create_engine(f"sqlite:///{db_path}")

In [ ]:
import os
print(os.path.exists(f"sqlite:///{db_path}"))

In [ ]:
# reflect an existing database into a new model
# reflect the tables

Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
print(Base.classes.keys())

In [82]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [83]:
# Create our session (link) from Python to the DB
from sqlalchemy.orm import Session
session = Session(engine)

In [ ]:
# query test
results = session.query(measurement.date, measurement.prcp).limit(5).all()
for result in results:
    print(result)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
most_recent_date = session.query(func.max(measurement.date)).scalar()
print("most recent date:", most_recent_date)

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results.
from datetime import datetime, timedelta

# Starting from the most recent data point in the database.
most_recent_date = datetime.strptime('2017-08-23', '%Y-%m-%d')

# Calculate the date one year from the last date in data set.
one_year_ago = most_recent_date - timedelta(days=365)
print("one year ago:", one_year_ago)

# Perform a query to retrieve the data and precipitation scores
prcp_data = session.query(measurement.date, measurement.prcp).filter(measurement.date >= one_year_ago).all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names.
prcp_df = pd.DataFrame(prcp_data, columns=['date', 'prcp'])
prcp_df = prcp_df.dropna()

# Ensure the 'date' column is correctly converted to datetime
prcp_df['date'] = pd.to_datetime(prcp_df['date'], format='%Y-%m-%d', errors='coerce')

# Sort the dataframe by date
prcp_df = prcp_df.sort_values(by='date')

# Use Pandas Plotting with Matplotlib to plot the data
import matplotlib.dates as mdates

plt.figure(figsize=(10, 6))
plt.bar(prcp_df['date'], prcp_df['prcp'], width=3)
plt.xlabel('Date')
plt.ylabel('Precipitation (in Inches)')
plt.title('Precipitation Over Last 12 Months')

# Format the x-axis to show months only
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
summary_stats = prcp_df['prcp'].describe()
print(summary_stats)

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
total_stations = session.query(func.count(station.station)).scalar()
print("total number of stations:", total_stations)

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
active_stations = (
    session.query(measurement.station, func.count(measurement.station)).group_by(measurement.station).order_by(func.count(measurement.station).desc()).all()
)
# List the stations and their counts in descending order.
print("most active stations:")
for station, count in active_stations:
    print(f"station: {station}, count: {count}")

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station = active_stations[0][0]

temperature_stats = (
    session.query(
        func.min(measurement.tobs), 
        func.max(measurement.tobs), 
        func.avg(measurement.tobs)   
    ).filter(measurement.station == most_active_station).all()
)

lowest_temp, highest_temp, avg_temp = temperature_stats[0]
print(f"lowest temperature: {lowest_temp}")
print(f"highest temperature: {highest_temp}")
print(f"average temperature: {avg_temp:.2f}")


In [ ]:
# Using the most active station id
most_active_station = active_stations[0][0]

# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
tobs_data = (
    session.query(measurement.tobs).filter(measurement.station == most_active_station).filter(measurement.date >= one_year_ago).all()
)

tobs_list = [temp[0] for temp in tobs_data]

plt.figure(figsize=(8, 6))
plt.hist(tobs_list, bins=12, edgecolor='black', alpha=0.7)
plt.title(f'Temperature Observations for Station {most_active_station} (Last 12 Months)')
plt.xlabel('Temperature (°F)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# Close Session

In [92]:
# Close Session
session.close()